# R Language API Wrapper, Prototype

In [1]:
getwd()
#setwd( paste( getwd(),'/ScrumSagaR',sep=''))
list.files()

[1] "/home/jovyan/work/notebooks"

[1] "admApiWrapper.ipynb"                      
 [2] "admProcessingTimeEstimation.ipynb"        
 [3] "admPythonTest.ipynb"                      
 [4] "admRlangWrapper.ipynb"                    
 [5] "admRPy2.ipynb"                            
 [6] "guideComparisonOfJsFrameworks.ipynb"      
 [7] "guideHelloWorld-Py.ipynb"                 
 [8] "guideHelloWorld-R.ipynb"                  
 [9] "guideTechniquesForSoftware.ipynb"         
[10] "guideTEMPLATE.ipynb"                      
[11] "guideWorkFlow.ipynb"                      
[12] "guideWorkingWithDifferentDataGroups.ipynb"
[13] "README.md"                                
[14] "ScrumSaga-Py"                             
[15] "ScrumSagaR"                               
[16] "Test"

In [1]:
# Acocunt information (must be manipulated on website: scrumsaga.com)
EMAIL='dev.team@mgmt-tech.org'
PASS='IMTorgTestUserPassword'

---
_Begin package__

In [2]:
library(httr)
library(jsonlite)


Account <- function(email, password){
    
    thisEnv <- environment()
    
    URL = "http://api.scrumsaga.com/v1"
    URL_LOCAL = '144.76.39.53:3001/v1'
    
    rteLogin = "/login"
    rteAcctData = '/acctData'
    rteAcctDiagram = '/acctDiagram'
    
    EMAIL <- email
    PASSWORD <- password
    TOKEN = 'TOKEN'

      me <- list(
          thisEnv = thisEnv,
          getToken = function(){
              return(get("TOKEN",thisEnv))
          },
          setToken = function(value){
              return(assign("TOKEN",value,thisEnv))
          },
          getUrl = function(){
              return(get("URL_LOCAL",thisEnv))
          },         
          api_status = function(){
              uri = URL_LOCAL
              resp = GET(uri)
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp
                  ),
                  class = "resp"
              )   
          },
          login = function() {
              uri = paste(URL_LOCAL, rteLogin, sep='')
              resp = POST(uri, body = list(email=EMAIL, password=PASSWORD), encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              me$setToken( paste('JWT',parsed$token ) )
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp
                  ),
                  class = "resp"
              )
          },
          check_data = function() {
              TOKEN = me$getToken()
              uri = paste(URL_LOCAL, rteAcctData, sep='')
              resp = POST(uri, add_headers(Authorization=TOKEN))
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          },
          check_diagram = function() {
              TOKEN = me$getToken()
              uri = paste(URL_LOCAL, rteAcctDiagram, sep='')
              resp = POST(uri, add_headers(Authorization=TOKEN))
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$msg,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          }
          
                
      )

      ## Define the value of the list within the current environment.
      assign('this',me,envir=thisEnv)

      ## Set the name for the class
      class(me) <- append(class(me),"Account")
      return(me)
}

In [92]:
userAcct <- Account(EMAIL, PASS)
userAcct$login()

$msg
[1] "passwords match"

$path
[1] "144.76.39.53:3001/v1/login"

$response
Response [http://144.76.39.53:3001/v1/login]
  Date: 2017-03-11 16:13
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 173 B


attr(,"class")
[1] "resp"

names(userAcct)
userAcct$getToken()
userAcct$getUrl()
userAcct$api_status()

userAcct$login()
userAcct$check_data()
userAcct$check_diagram()

In [7]:
packageVersion('httr')

[1] ‘1.2.1’

In [86]:
payload=list(
namespace = 'IMTorgTestCode',
email = 'information@mgm-tech.org',
repo = 'testprj_Java_aSimple',
selection = 'all'
)

In [88]:
print( payload )

$namespace
[1] "IMTorgTestCode"

$email
[1] "information@mgm-tech.org"

$repo
[1] "testprj_Java_aSimple"

$selection
[1] "all"



# INPUTS
argGroups = c('project', 'size', 'author', 'entity_structure', 'entity_characteristic', 'complexity', 'relation', 'error','quality','tag','process_log')
argSelection = c('current', 'all')
rtePathLoadGroup = '/load'
rtePathExtract = '/extract'

ACCT = userAcct
REPO_NAMESPC = 'IMTorg'
REPO_EMAIL = 'information@mgmt-tech.org'
REPO_NAME = 'testprj_Java_aSimple'

In [6]:
# EXTRACT
TOKEN = userAcct$getToken()
uri = '144.76.39.53:3001/v1/extract'
#payload = list(namespace=REPO_NAMESPC, email=REPO_EMAIL, repo=REPO_NAME, select='current')

resp = POST(uri, add_headers(Authorization=TOKEN), body=payload, encode = "json" )
resp
jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)

Response [http://144.76.39.53:3001/v1/extract]
  Date: 2017-03-11 15:07
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 39 B


$message
[1] "No need to process, again"

In [99]:
library(httr)
library(jsonlite)


Project <- function(account, repo_namespace, repo_email, repo_name){
    
    thisEnv <- environment()
    
    # system (class) config
    argGroups = c('project', 'size', 'author', 'entity_structure', 'entity_characteristic', 'complexity', 'relation', 'error','quality','tag','process_log')
    argSelection = c('current', 'all')
    # API paths
    rtePathLoadGroup = '/load/'
    rtePathExtract = '/extract'
    
    ACCT = account
    PAYLOAD=list(
        namespace = repo_namespace,
        email = repo_email,
        repo = repo_name
    )

      me <- list(
          thisEnv = thisEnv,
          getGroups = function(){
              return(get("argGroups",thisEnv))
          },
          getAcct = function(){
              return(get("ACCT",thisEnv))
          }, 
          getPayload = function(){
              return(get("PAYLOAD",thisEnv))
          },
          extract = function(select='all'){
              ACCT = me$getAcct()
              PAYLOAD = me$getPayload()
              PAYLOAD$selection = select
              TOKEN = ACCT$getToken()
              rteExtractData = 'extract'
              uri = paste( ACCT$getUrl(), rteExtractData, sep='/')
              resp = POST(uri, add_headers(Authorization=TOKEN), body=PAYLOAD , encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              parsed = jsonlite::fromJSON(content(resp, "text"), simplifyVector = FALSE)
              structure(
                  list(
                      msg = parsed$message,
                      path = uri,
                      response = resp,
                      data = parsed$data
                  ),
                  class = "resp"
              )
          },
          load_group = function(group='project') {
              ACCT=me$getAcct()
              PAYLOAD=me$getPayload()
              TOKEN = ACCT$getToken()
              rteLoadData = 'load'
              uri = paste( ACCT$getUrl(), rteLoadData, group, sep='/')
              resp = POST(uri, add_headers(Authorization=TOKEN), body=PAYLOAD , encode = "json")
              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              data = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)
            return(data)
          }
        
      )

      ## Define the value of the list within the current environment.
      assign('this',me,envir=thisEnv)

      ## Set the name for the class
      class(me) <- append(class(me),"Project")
      return(me)
}

In [100]:
jSimple = Project(account=userAcct, repo_namespace='IMTorgTestCode', repo_emai='information@mgm-tech.org', repo_name='testprj_Java_aSimple')
names(jSimple)
jSimple$getGroups()
#jSimple$getAcct()

[1] "thisEnv"    "getGroups"  "getAcct"    "getPayload" "extract"   
[6] "load_group"

[1] "project"               "size"                  "author"               
 [4] "entity_structure"      "entity_characteristic" "complexity"           
 [7] "relation"              "error"                 "quality"              
[10] "tag"                   "process_log"

In [101]:
jSimple$extract(select='all')

$msg
[1] "No need to process, again"

$path
[1] "144.76.39.53:3001/v1/extract"

$response
Response [http://144.76.39.53:3001/v1/extract]
  Date: 2017-03-11 16:14
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 39 B


$data
NULL

attr(,"class")
[1] "resp"

In [102]:
Prj = jSimple$load_group(group='size')
dim(Prj)

[1] 14 14

# Archive

In [7]:
#LOAD
TOKEN = userAcct$getToken()
rte2 = '/load/project'
uri = paste(userAcct$getUrl(), rte2, sep='')

resp = POST(uri, add_headers(Authorization=TOKEN), body=payload, encode='json')
resp
Prj = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)

dim(Prj)

Response [http://144.76.39.53:3001/v1/load/project]
  Date: 2017-03-11 15:07
  Status: 200
  Content-Type: application/json; charset=utf-8
  Size: 9.69 kB


[1] 14 27

In [15]:
        load_group = function(ACCT, PAYLOAD, GROUP) {
              TOKEN = ACCT$getToken()
              rteLoadData = 'load'
              uri = paste( ACCT$getUrl(), rteLoadData, GROUP, sep='/')
              payload = PAYLOAD
              resp = POST(uri, add_headers(Authorization=TOKEN), body=payload , encode = "json")

              if (http_type(resp) != "application/json") {
                  stop("API did not return json", call. = FALSE)
              }
              data = data.frame(jsonlite::fromJSON(content(resp,'text'))$data)
            return(data)
          }


Prj = load_group(ACCT=userAcct, PAYLOAD=payload, GROUP='error')
dim(Prj)

[1] 701   8